In [1]:
import sklearn
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import MiniBatchKMeans
import matplotlib.pyplot as plt
from eli5 import show_weights, show_prediction
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.linear_model import LogisticRegression


#custom
from helper import DataProcessing
from ml_metrics import evaluate_model, multiclass_logloss
from plotting import plot_tsne

/home/didier/anaconda3/envs/c19biohack_ml/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/didier/anaconda3/envs/c19biohack_ml/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.
/home/didier/anaconda3/envs/c19biohack_ml/lib/python3.7/site-packages/tensorfl

In [2]:
# set K-mer length here
kmer_list = range(3,9)

In [3]:
#K-means clustering
def run_kmeans(k):
    class_dict = dict(zip(range(k), [f'cluster_{i}' for i in range(k)]))
    kmeans = MiniBatchKMeans(n_clusters=k,verbose=0, batch_size=100,random_state=101)
    ytrain = kmeans.fit_predict(xtrain_ctv)
    ytest = kmeans.fit_predict(xtest_ctv)
    return ytrain, ytest, class_dict

In [4]:
orf1 = DataProcessing('coronavirus_orf1ab.fasta', 'coronavirus_orf1ab_meta.csv')

In [5]:
%%time
np.random.seed(seed = 101)
loss_dict_k_4 = {}
for kmer in kmer_list:
    amino_df = orf1.get_amino_df(kmer)
#     print(amino_df.shape,"original shape")

    # get rid of duplicates
    amino_df.drop_duplicates(subset='Accession', keep=False, inplace=True)
#     print(amino_df.shape,"filtered shape")
    mask = np.random.rand(len(amino_df)) < 0.8
    train_df = amino_df[mask]
    test_df = amino_df[~mask]
    xtrain = train_df['seq_offset_0'].values
    xtest = test_df['seq_offset_0'].values
#     print(f'Size of the test df: {len(test_df)}. Size of the tain df: {len(train_df)}.')
    #vectorize
    ctv = CountVectorizer(analyzer='char', ngram_range=(kmer, kmer), lowercase=False) # kmer: k-mer length

    ctv.fit(list(xtrain)+list(xtest))
    xtrain_ctv = ctv.transform(xtrain)
    xtest_ctv = ctv.transform(xtest)
    ytrain, ytest, class_dict = run_kmeans(4)
    #fit logistic regression on CountVectorizer
    
    clf = LogisticRegression(C=1.0, max_iter=4000,random_state=101 , n_jobs=-1)
    clf.fit(xtrain_ctv, ytrain)
    predictions = clf.predict_proba(xtest_ctv)
    loss_dict_k_4["%i kmer"%kmer]= {"logloss":multiclass_logloss(ytest, predictions),
                      "test set size":test_df.shape[0] , 
                      "train set size":train_df.shape[0] }
#     print ("************ kmer = {} *********".format(kmer))
#     print (multiclass_logloss(ytest, predictions))
#     evaluate_model(predictions, ytest, class_dict)
#     amino_df.head()

CPU times: user 16min 27s, sys: 7.92 s, total: 16min 35s
Wall time: 14min 1s


In [6]:
%%time
np.random.seed(seed = 101)
loss_dict_k_5 = {}
for kmer in kmer_list:
    amino_df = orf1.get_amino_df(kmer)
#     print(amino_df.shape,"original shape")

    # get rid of duplicates
    amino_df.drop_duplicates(subset='Accession', keep=False, inplace=True)
#     print(amino_df.shape,"filtered shape")
    mask = np.random.rand(len(amino_df)) < 0.8
    train_df = amino_df[mask]
    test_df = amino_df[~mask]
    xtrain = train_df['seq_offset_0'].values
    xtest = test_df['seq_offset_0'].values
#     print(f'Size of the test df: {len(test_df)}. Size of the tain df: {len(train_df)}.')
    #vectorize
    ctv = CountVectorizer(analyzer='char', ngram_range=(kmer, kmer), lowercase=False) # kmer: k-mer length

    ctv.fit(list(xtrain)+list(xtest))
    xtrain_ctv = ctv.transform(xtrain)
    xtest_ctv = ctv.transform(xtest)
    ytrain, ytest, class_dict = run_kmeans(5)
    #fit logistic regression on CountVectorizer
    
    clf = LogisticRegression(C=1.0, max_iter=4000,random_state=101 , n_jobs=-1)
    clf.fit(xtrain_ctv, ytrain)
    predictions = clf.predict_proba(xtest_ctv)
    loss_dict_k_5["%i kmer"%kmer]= {"logloss":multiclass_logloss(ytest, predictions),
                      "test set size":test_df.shape[0] , 
                      "train set size":train_df.shape[0] }
#     print ("************ kmer = {} *********".format(kmer))
#     print (multiclass_logloss(ytest, predictions))
#     evaluate_model(predictions, ytest, class_dict)
#     amino_df.head()

CPU times: user 11min 36s, sys: 6.79 s, total: 11min 43s
Wall time: 10min 6s


In [7]:
import pandas as pd 

In [10]:
df_4K = pd.DataFrame.from_dict(loss_dict_k_4).T

In [11]:
df_5K = pd.DataFrame.from_dict (loss_dict_k_5).T

In [13]:
df_4K.join(df_5K, lsuffix='_k4', rsuffix='_k5')

,logloss_k4,test set size_k4,train set size_k4,logloss_k5,test set size_k5,train set size_k5
3 kmer,13.404110,457.0,1927.0,5.268189,457.0,1927.0
4 kmer,5.584802,511.0,1873.0,3.821062,511.0,1873.0
5 kmer,10.268704,475.0,1909.0,8.464424,475.0,1909.0
6 kmer,9.306516,488.0,1896.0,13.745919,488.0,1896.0
7 kmer,8.984164,437.0,1947.0,14.066950,437.0,1947.0
8 kmer,8.209397,467.0,1917.0,13.805108,467.0,1917.0
